In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
import plotly.express as px
import numpy as np
my_css_link = 'https://codepen.io/chriddyp/pen/bWLwgP.css'


# Create a dash application
app = dash.Dash(external_stylesheets=[my_css_link])


# Read the  data into pandas dataframe
df = px.data.gapminder()
df.head(10)

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
5,Afghanistan,Asia,1977,38.438,14880372,786.113360,AFG,4
6,Afghanistan,Asia,1982,39.854,12881816,978.011439,AFG,4
7,Afghanistan,Asia,1987,40.822,13867957,852.395945,AFG,4
8,Afghanistan,Asia,1992,41.674,16317921,649.341395,AFG,4
9,Afghanistan,Asia,1997,41.763,22227415,635.341351,AFG,4


In [2]:
df['lifeExp'].max()

82.603

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   country    1704 non-null   object 
 1   continent  1704 non-null   object 
 2   year       1704 non-null   int64  
 3   lifeExp    1704 non-null   float64
 4   pop        1704 non-null   int64  
 5   gdpPercap  1704 non-null   float64
 6   iso_alpha  1704 non-null   object 
 7   iso_num    1704 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 106.6+ KB


In [4]:
# Build dash app layout
app.layout = html.Div([
                html.H1('Exploring gdpPercap and Life Expectancy', 
                    style={'textAlign': 'center', 'color': '#503D36',
                    'font-size': 30}),
                html.Div([
                    html.Div([
                        html.Pre(children="Select year", style={"fontSize":"150%",'textAlign': 'center'}),
                        dcc.Slider(
                                    id='my-slider10',
                                    min=df.year.min(),
                                    max=df.year.max(),
                                    step=None,
                                    value=df.year.min(),
                                    marks={str(year): str(year) for year in df.year.unique()}),
                        html.Pre(children="Select continent", style={"fontSize":"150%",'textAlign': 'center'}),
                        dcc.Dropdown(
                                id='my-dropdown10',
                                options=[{'label':con, 'value':con} for con in df.continent.unique()
                                ],
                                value='Africa',
                                style={'width':"100%"}),
                        html.Button('Submit', id='submit-val1', n_clicks=0),
                        dcc.Graph(id='myGraph10',figure={})
                    ], className='six columns'),
                    
                    html.Div([
                        html.Pre(children="Select country", style={"fontSize":"150%",'textAlign': 'center'}),
                        html.Br(),
                        dcc.Dropdown(
                                id='my-dropdown11',
                                options=[{'label':con, 'value':con} for con in df.country.unique()
                                ],
                                value='Egypt',
                                style={'width':"100%"}),

                        html.Br(),
                        html.Br(),
                        html.Pre(children="gdpPercap for each country over years", style={"fontSize":"150%",'textAlign': 'center' }),
                        dcc.Graph(id='myGraph11',figure={})
                    ], className='six columns'),
                ],className= 'row'),
                
                html.Div([
                    html.Div([
                        html.Pre(children="lifeExp for countries and continents", style={"fontSize":"150%",'textAlign': 'center' }),
                        dcc.Graph(id='myGraph20',figure={})
                    ], className='six columns'),
                    
                    html.Div([
                        html.Pre(children="gdpPercap over years", style={"fontSize":"150%",'textAlign': 'center' }),
                        dcc.Graph(id='myGraph21',figure={})
                    ], className='six columns'),
                ],className= 'row')
                
])


# Callback decorator
@app.callback(
            Output(component_id='myGraph10', component_property='figure'),
            Output('myGraph11', 'figure'),
            Output('myGraph20', 'figure'),
            Output('myGraph21', 'figure'),
            State('my-slider10','value'),
            State('my-dropdown10', 'value'),
            Input('submit-val1', 'n_clicks'),
            Input('my-dropdown11', 'value'),

            
)
# Computation to callback function and return graph
def get_graph(year10, cont10, n_clicks1, cont11):
    
    fig1 = px.scatter(df[(df.year==year10) & (df.continent==cont10)], x='gdpPercap', y='lifeExp',log_x=True,hover_name='country',template='plotly_dark',
           color='country', size='pop', size_max=50, labels={'gdpPercap':'gdp per capita', 'lifeExp':'life Expectancy'}, title='exp per gdpPerCap')
    fig2 = px.line(df[df.country==cont11], x='year', y='gdpPercap',template='plotly_dark')
    fig3 = px.sunburst(df, path=['continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df['lifeExp'], weights=df['pop']))
    fig4 = px.pie(df, names='year', values='gdpPercap')
    
    return fig1, fig2, fig3, fig4





# Run the app
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Oct/2021 11:16:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 11:16:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 11:16:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 11:16:10] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 11:16:10] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 11:16:10] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 11:16:10] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 11:16:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 11:48:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 11:48:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 11:48:58] "POST /_dash-